In [ ]:
! pip install -q autoawq accelerate

In [ ]:
!pip install -q flash-attn --no-build-isolation

In [ ]:
# Mount you google drive
from google.colab import drive
import os
drive.mount("/content/drive/")

In [4]:
# Login to the HuggingFace hub
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('Add your HuggingFace token'))

In [ ]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

# Specify the paths
model_path = "Add your unquantized model repo from HuggingFace (e.g. Qwen/Qwen2.5-Omni-7B)"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

In [6]:
quant_path = "Add the path to where you want to save the model"

# Quantization configuration
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

In [ ]:
model = AutoAWQForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

In [ ]:
# Print the maximum sequence length
print(f"Max position embeddings: {model.config.max_position_embeddings}")

In [ ]:
# Quantize the model
model.quantize(
    tokenizer,
    quant_config=quant_config,
)

In [ ]:
# Save quantized model
model.save_quantized(quant_path, safetensors=True)
tokenizer.save_pretrained(quant_path)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path=quant_path,
    repo_id="Add your AWQ HuggingFace repo id",
    repo_type="model",
)